In [1]:
##############################
# probs0.json - take 2
##############################

import requests
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import random

tstart = time.time()
print(tstart)

api_url = 'https://api.hooktheory.com/v1/'
login = {"username": "jl8429",
         "password": ""}

r = requests.post(api_url+'users/auth', data=login)
#print('r', r)
#print(r.json())
'''
{'id': 136206, 'username': 'myUsername', 'activkey': 'myKey'}
'''

activkey = '0e11e1d65a021d0a883f309f813d74ad'
header = {"Authorization": "Bearer "+activkey}

#with open('chordProbs3new.json', "r") as f:
with open('newprobs0.json', "r") as f:
    data3 = json.load(f)

#################################
# RELOAD big_dict and genre_dict and histo
#################################

# with open('probs0genre.json', 'r') as fg:
#     big_dict = json.load(fg)

# genre_dict = {}

# for bd in big_dict:
#     if big_dict[bd]:
#         for song in big_dict[bd]:
#             genre_dict[song['url']] = song['genre']

# for gd in genre_dict:
#     print(gd, genre_dict[gd])
#     break

######## OR ########

try:
    genre_dict # don't overwrite it if it exists for future jsons
except:
    genre_dict = {}
big_dict = {}

histo = [0]*5000

#################################
# continue
#################################

filename3se = 'newprobs0e.txt'
f2 = open(filename3se, 'w')

j = 1
totals = str(len(data3))
print(totals)

driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")

for d in data3:
    chord1, chord2, chord3, chord4 = d['child_path'].split(",")
    mykey = str(chord1)+','+str(chord2)+','+str(chord3)+','+str(chord4)
    print(str(j)+'/'+totals, mykey)
    song_list = []
    
    # don't overwrite what we've done already
    if not big_dict.get(mykey, False):
        try:
            myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
            page = 1
            keepGoing = True
            #while myData.content and myData.content != b'No songs match this chord progressionnull':
            while keepGoing:
                # can't parse if it's this byte string
                if myData.content != b'No songs match this chord progressionnull':
                    #print('page', page)
                    # parse each song
                    try:
                        my_json = myData.json()
                        songs = len(my_json)
                        #print('songs len', songs)
                        if songs:
                            # for each song, pull the genre, add it to the genre_dict and big_dict, then write it to the file
                            for i in range(songs):
                                url = my_json[i]['url'].split("#")[0]
                                #print(url)
                                # check if already in genre_dict to save us from having to load the page slowly and parse
                                if url in genre_dict:
                                    #print(url, genre_dict[url])
                                    my_json[i]['genre'] = genre_dict[url]
                                # if not in dict, we need to parse page and add to dict
                                else:
                                    driver.get(url)
                                    html = driver.page_source
                                    soup = BeautifulSoup(html, 'html.parser')
                                    try:
                                        #genre = soup.find_all(['multiselect'])[0].find('a').contents[0]
                                        # some songs don't have a genre and some have multiple genres...
                                        genres = soup.find_all(['multiselect'])[0].find_all('a')
                                        gk = 0
                                        glist = []
                                        for g in genres:
                                            # .find_all() will return a lot of garbage, but the genres will have a length of 1
                                            if len(g) > 1:
                                                # if even the first is garbage, it has no genre
                                                if gk == 0:
                                                    my_json[i]['genre'] = ["Unknown"]
                                                    genre_dict[url] = ["Unknown"]
                                                break
                                            else:
                                                glist.append(g.contents[0])
                                            gk += 1
                                        my_json[i]['genre'] = glist
                                        genre_dict[url] = glist
                                    except:
                                        print('3. Unable to extract genre for', url)
                                        my_json[i]['genre'] = ["Unknown"]
                                        genre_dict[url] = ["Unknown"]
                                        # sleep for 2 seconds in case it was a "Too Many Requests" issue
                                        time.sleep(2)

                                    # this is not where this block belongs
                                    # nesting it under "else" above made failures generate "[]" and 
                                    #    only appended new songs to song_list, ignoring those already in genre_dict
#                                     #driver.close()
#                                     # clean up and append song to list
#                                     my_json[i].pop('section', None)
#                                     my_json[i]['url'] = url
#                                     #f.write('\n'+str(my_json[i])+',')
#                                     song_list.append(my_json[i])

                                # this is where the above block really belongs 
                                #     so song_list gets every song, even if the url is in genre_dict
                                #driver.close()
                                # clean up and append song to list
                                my_json[i].pop('section', None)
                                my_json[i]['url'] = url
                                #f.write('\n'+str(my_json[i])+',')
                                #print(my_json[i])
                                song_list.append(my_json[i])

                            if songs == 20:
                                #keepGoing = True
                                page += 1
                                myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                            else:
                                # update histogram of how many songs per chord progression
                                index = page*20 - 20 + songs
                                print('histo1', index)
                                histo[index] += 1
                                keepGoing = False
                        else:
                            # update histogram of how many songs per chord progression
                            index = page*20 - 20
                            print("histo2", index)
                            histo[index] += 1
                            keepGoing = False

                    except:
                        # for some reason, page=20 doesn't work on 1-5-6-4... there are other data corruptions too
                        print('2. Unable to extract for', chord1, chord2, chord3, chord4, "page", page)
                        f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
                        f2.close()
                        f2 = open(filename3se, 'a')

                        # sleep for 2 seconds in case it was a "Too Many Requests" issue
                        time.sleep(2)

                        page += 1
                        myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                else:
                    print("No songs match this chord progression", mykey)
                    keepGoing = False

        except:
            print('1. Unable to extract for', chord1, chord2, chord3, chord4)
            f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
            f2.close()
            f2 = open(filename3se, 'a')

            # sleep for 2 seconds in case it was a "Too Many Requests" issue
            time.sleep(2)
        
        big_dict[mykey] = song_list
    #f.close()
    #f = open(filename3s, 'a')
    #time.sleep(1.1)
    j += 1
    # restart the driver every 100 chord progressions
    if j%100==0:
        driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")
    
f2.close()

filename3s = 'newprobs0genre.json'
with open(filename3s, "w") as f:
    json.dump(big_dict, f)

tend = time.time()
print(tend)
#f.close()
print(tend-tstart)

print(histo)
with open("histo0.txt", "w") as f:  
    f.write(str(histo))

1614991980.2589695
500
1/500 264,6,264,6
histo1 2
2/500 56,46,16,265
histo1 1
3/500 17,47,6,56/2
histo1 1
4/500 46,1,16,164
histo1 1
5/500 1,164,b6,b7
histo1 2
6/500 4,1,27,3
histo1 1
7/500 16,3,5,1
histo1 1
8/500 4,2,16,265
histo1 1
9/500 77,57/6,6,b5
histo1 1
10/500 1,67,57,1
histo1 1
11/500 4,57,1,b46
histo1 1
12/500 1,5,464,1
histo1 1
13/500 664,6,664,364
histo1 1
14/500 7/6,6,664,6
histo1 1
15/500 6,3,6,57/6
histo1 1
16/500 5,3,47,6
histo1 1
17/500 37,6,47,57
histo1 1
18/500 4,6,3,5
histo1 5
19/500 4,47,1,5
histo1 1
20/500 4,2,5/6,4
histo1 4
21/500 5/5,5/2,5/5,5/2
histo1 1
22/500 4,5,664,164
histo1 1
23/500 37,D37,Y27,17
histo1 1
24/500 5/2,2,5/2,2
histo1 3
25/500 16,27,2,5
histo1 1
26/500 542,16,D442,27
histo1 1
27/500 5/2,4,56,4
histo1 1
28/500 5/2,4,Y7,4
histo1 1
29/500 5/6,6,5/5,5
histo1 1
30/500 2,242,565,5
histo1 2
31/500 7,5,6,5
histo1 2
32/500 5,3,L7,5/2
histo1 1
33/500 1,56,56/6,664
histo1 1
34/500 664,2,7/6,6
histo1 1
35/500 57,243,57,243
histo1 1
36/500 542,4,542,4
hist

histo1 5
291/500 47,1,37,6
histo1 4
292/500 3,2,26,564/6
histo1 1
293/500 4,37,M67,27
histo1 1
294/500 4,1,67,3
histo1 1
295/500 6,36,L4,L3
histo1 1
296/500 664,5/6,56/2,2
histo1 1
297/500 1,4,5/6,4
histo1 3
298/500 664,6,76,6
histo1 1
299/500 57,164,57,1
histo1 3
300/500 564,1,16,2
histo1 1
301/500 47,b465,17,27
histo1 1
302/500 17,57/6,67,27
histo1 1
303/500 2,b7,66,2
histo1 1
304/500 5,56/6,67,643
histo1 1
305/500 6,2,37,6
histo1 1
306/500 564,464,56,6
histo1 1
307/500 1,5,b7,4
histo1 30
308/500 4,1,364,b7
histo1 1
309/500 57,1,16,6
histo1 1
310/500 b7,1,5,1
histo1 1
311/500 47,5,6,L7
histo1 1
312/500 6,D6,6,b46
histo1 1
313/500 2,5,17,464
histo1 1
314/500 57,6,5/5,6
histo1 1
315/500 27,57,67,4
histo1 1
316/500 265,3,2,3
histo1 1
317/500 47,165,67,27
histo1 1
318/500 1,4,5,27
histo1 2
319/500 47,27,664,4
histo1 1
320/500 1,16,4,57/2
histo1 1
321/500 37,4,67,17
histo1 1
322/500 3,5/3,3,5/5
histo1 1
323/500 37,4,5,37
histo1 4
324/500 1,564,464,4
histo1 2
325/500 6,664,6,2
histo1 3
326

In [ ]:
# there's a dash before the page, should be /i-wont-back-down instead of /-i-wont-back-down
339/2000 5,1,6,5
3. Unable to extract genre for http://www.hooktheory.com/theorytab/view/tom-petty-and-the-heartbreakers/-i-wont-back-down
histo1 47

# there's a dash after the page, should be /jack-sparrow instead of /jack-sparrow-
917/2000 6,5,6,5
2. Unable to extract for 6 5 6 5 page 1
3. Unable to extract genre for http://www.hooktheory.com/theorytab/view/hans-zimmer/jack-sparrow-
    
# some don't have songs...
'b7 M17 b7 5/6' and '5 542 16 564'
b'No songs match this chord progressionnull'
# actually that above error is only because we used spaces instead of commas. they are still due to:
b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'

In [4]:
# discovered code was only saving new songs not already in genre_dict...
mykey = '5 542 16 564'
page = 1
myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
#myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
print(myData.content)
#print(len(myData.json()))
#print(myData.json())

b'No songs match this chord progressionnull'


In [5]:
# print(histo)
# with open("histo0.txt", "w") as f:  
#     f.write(str(histo))

[0, 371, 58, 21, 14, 12, 2, 1, 2, 3, 1, 2, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 